## 0. Libraries import

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# imports 
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
#train test split
from sklearn.model_selection import train_test_split
# Hyperparameters selection
from sklearn.model_selection import RandomizedSearchCV
# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
# models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
# error
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

## 1. Data import

### 1.1 Impor train data

In [3]:
df_diamonds_train=pd.read_csv('../data/diamonds_train.csv')
df_diamonds_train.pop("Unnamed: 0") #dropped an unnecessary column
df_diamonds_train

,index_id,depth,table,x,y,z,price,carat,cut,color,clarity,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,505,0.32,Very Good,H,VS2,Kimberly
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65,2686,0.71,Fair,G,VS1,Las Vegas
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,738,0.41,Good,D,SI1,Kimberly
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,4882,1.02,Ideal,G,SI1,Dubai
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43,10070,1.34,Ideal,G,VS1,Antwerp
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73,12615,2.02,Good,F,SI2,Madrid
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,4.01,5457,1.01,Ideal,H,SI1,Kimberly
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,61.9,54.3,4.45,4.47,2.76,456,0.33,Ideal,J,VS1,Kimberly


### 1.2 Impor test data

In [4]:
df_diamonds_test=pd.read_csv('../data/diamonds_test.csv')
df_diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


## 2. Data preparation (training set)

### 2.1 Defining numerical and categorical features

In [5]:
# Excluding city (with trees sometimes is better to keep variables even if they are correlated)
num_features_list=['x','y','z','depth','table','carat']
cat_features_list=['cut','color','clarity','city']
target='price'
features_list=['x','y','z','depth','table','carat','cut','color','clarity','city']

In [6]:
# 2. Cross target encoding
# Mean
cut_color_encoding = df_diamonds_train.groupby(['cut','color'])['price'].mean().to_dict()
df_diamonds_train['cut_color_encoding'] = df_diamonds_train.set_index(['cut',
                                                                       'color']).index.map(cut_color_encoding.get).astype(float)
cut_clarity_encoding = df_diamonds_train.groupby(['cut','clarity'])['price'].mean().to_dict()
df_diamonds_train['cut_clarity_encoding'] = df_diamonds_train.set_index(['cut',
                                                                       'clarity']).index.map(cut_clarity_encoding.get).astype(float)
color_clarity_encoding = df_diamonds_train.groupby(['color','clarity'])['price'].mean().to_dict()
df_diamonds_train['color_clarity_encoding'] = df_diamonds_train.set_index(['color',
                                                                           'clarity']).index.map(color_clarity_encoding.get).astype(float)

In [7]:
features_list_encoding=['x','y','z','depth','table','carat','cut','color','clarity','city'
                        ,'cut_color_encoding','cut_clarity_encoding','color_clarity_encoding']

### 3.5 One hot encoding

In [8]:
# 3. Defining features y target
#features_list_volume=['x', 'y', 'z', 'depth', 'table', 'carat', 'cut', 'color', 'clarity','volume']
X=df_diamonds_train[features_list_encoding]
y=df_diamonds_train['price']

In [9]:
for column in cat_features_list:
    X[column]=X[column].astype('category')
    X[column]=X[column].cat.codes

In [10]:
X

,x,y,z,depth,table,carat,cut,color,clarity,city,cut_color_encoding,cut_clarity_encoding,color_clarity_encoding
0,6.83,6.79,4.25,62.4,58.0,1.21,3,6,5,2,6376.983740,4577.414201,5260.516874
1,4.35,4.38,2.75,63.0,57.0,0.32,4,4,5,3,4541.993464,4277.315195,4686.621902
2,5.62,5.53,3.65,65.5,55.0,0.71,0,3,4,4,4473.021368,4151.344262,4153.819453
3,4.68,4.72,3.00,63.8,56.0,0.41,1,0,2,3,3311.693089,3601.061697,2970.951792
4,6.55,6.51,3.95,60.5,59.0,1.02,2,3,2,2,3751.364560,3759.057457,3787.558804
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,7.10,7.04,4.43,62.7,57.0,1.34,2,3,4,1,3751.364560,3484.675277,4153.819453
40451,8.31,8.25,4.73,57.1,60.0,2.02,1,2,3,7,3477.504518,4562.165025,4511.178542
40452,6.37,6.42,4.01,62.7,56.0,1.01,2,4,2,3,3873.835802,3759.057457,5042.731524
40453,4.45,4.47,2.76,61.9,54.3,0.33,2,6,4,3,4854.274311,3484.675277,4781.833741


### 3.7 Define train and validation

In [11]:
# 5.Splitting train and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train.shape

(32364, 13)

In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32364 entries, 32121 to 15795
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   x                       32364 non-null  float64
 1   y                       32364 non-null  float64
 2   z                       32364 non-null  float64
 3   depth                   32364 non-null  float64
 4   table                   32364 non-null  float64
 5   carat                   32364 non-null  float64
 6   cut                     32364 non-null  int8   
 7   color                   32364 non-null  int8   
 8   clarity                 32364 non-null  int8   
 9   city                    32364 non-null  int8   
 10  cut_color_encoding      32364 non-null  float64
 11  cut_clarity_encoding    32364 non-null  float64
 12  color_clarity_encoding  32364 non-null  float64
dtypes: float64(9), int8(4)
memory usage: 2.6 MB


## 4. Data preparation (test set)

## 5. Feature engineering (test set) 

In [14]:
# 0. Adapting categorical features for validation model
X_test=df_diamonds_test[features_list]

In [15]:
# 2. Cross target encoding
# Mean
cut_color_encoding = df_diamonds_train.groupby(['cut','color'])['price'].mean().to_dict()
X_test['cut_color_encoding'] = X_test.set_index(['cut','color']).index.map(cut_color_encoding.get).astype(float)
cut_clarity_encoding = df_diamonds_train.groupby(['cut','clarity'])['price'].mean().to_dict()
X_test['cut_clarity_encoding'] = X_test.set_index(['cut','clarity']).index.map(cut_clarity_encoding.get).astype(float)
color_clarity_encoding = df_diamonds_train.groupby(['color','clarity'])['price'].mean().to_dict()
X_test['color_clarity_encoding'] = X_test.set_index(['color','clarity']).index.map(color_clarity_encoding.get).astype(float)

### 3.6 Change categorical variables to code

In [16]:
for column in cat_features_list:
    X_test[column]=X_test[column].astype('category')
    X_test[column]=X_test[column].cat.codes

In [17]:
X_test

,x,y,z,depth,table,carat,cut,color,clarity,city,cut_color_encoding,cut_clarity_encoding,color_clarity_encoding
0,5.82,5.89,3.67,62.7,60.0,0.79,4,2,2,0,3790.583636,3959.649629,3679.771031
1,6.81,6.89,4.18,61.0,57.0,1.20,2,6,4,10,4854.274311,3484.675277,4781.833741
2,7.38,7.32,4.57,62.2,61.0,1.57,3,4,2,3,5241.424619,4494.566378,5042.731524
3,6.09,6.13,3.90,63.8,54.0,0.90,4,2,2,3,3790.583636,3959.649629,3679.771031
4,5.05,5.09,3.19,62.9,58.0,0.50,4,2,4,0,3790.583636,3763.722052,3659.599024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,5.35,5.32,3.30,61.9,56.0,0.57,2,1,2,0,2624.818649,3759.057457,3132.153118
13481,5.71,5.73,3.56,62.2,55.0,0.71,2,5,5,8,4409.012354,3214.791678,5714.858295
13482,5.75,5.71,3.53,61.6,55.0,0.70,2,2,4,11,3284.470003,3484.675277,3659.599024
13483,5.85,5.89,3.45,58.8,57.0,0.70,4,2,3,10,3790.583636,5017.045820,4511.178542


In [18]:
X_test.shape

(13485, 13)

In [49]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   x                       13485 non-null  float64
 1   y                       13485 non-null  float64
 2   z                       13485 non-null  float64
 3   depth                   13485 non-null  float64
 4   table                   13485 non-null  float64
 5   carat                   13485 non-null  float64
 6   cut                     13485 non-null  int8   
 7   color                   13485 non-null  int8   
 8   clarity                 13485 non-null  int8   
 9   city                    13485 non-null  int8   
 10  cut_color_encoding      13485 non-null  float64
 11  cut_clarity_encoding    13485 non-null  float64
 12  color_clarity_encoding  13485 non-null  float64
dtypes: float64(9), int8(4)
memory usage: 1001.0 KB


## Model definition - RandomForestRegressor - with Random Hyperparameter Grid

RandomForestRegressor: multiple trees in paralel changing samples and convining diferrent features (overfitting when the tree is big and good to reduce error variance)

Main Parameters:
   - bootstrap -> method for sampling data points (TRUE bagging and FALSE pasting, with/without replacement)
   - n_estimators -> number of trees in the foreset
   - max_depth -> max number of levels in each decision tree
   - max_features -> max number of features considered for splitting a node
   - ccp_alpha ->
   - criterion ->
   - max_leaf_nodes -> max number of solution nodes 
   - max_samples ->
   - min_impurity_decrease ->
   - min_samples_leaf -> min number of data points allowed in a leaf node
   - min_samples_split -> min number of data points placed in a node before the node is split
   - min_weight_fraction_leaf
   - n_estimators -> number of trees in the foreset
   - n_jobs 
   - oob_score 
   - random_state
   - verbose
   - warm_start   

In [19]:
# 1. XGBRegressor 
model = XGBRegressor(n_estimators=200,colsample_bylevel=1,colsample_bynode=1,
                     colsample_bytree=0.8,reg_alpha=1, reg_lambda=1,gamma=0,learning_rate=0.1, random_state=42)
hyperparameters = model.get_params()
print(type(model), '\n')
print('Model hyperparameters:', hyperparameters, '\n')

<class 'xgboost.sklearn.XGBRegressor'> 

Model hyperparameters: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.8, 'enable_categorical': False, 'gamma': 0, 'gpu_id': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_delta_step': None, 'max_depth': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 200, 'n_jobs': None, 'num_parallel_tree': None, 'predictor': None, 'random_state': 42, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None} 



## Model training with validation

In [20]:
%%time
# 1. XGBRegressor early_stopping_rounds=40
# Model training
model.fit(X_train, y_train,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:5054.71387	validation_1-rmse:5092.72266
[1]	validation_0-rmse:4566.81201	validation_1-rmse:4602.24853
[2]	validation_0-rmse:4128.76318	validation_1-rmse:4164.24609
[3]	validation_0-rmse:3735.40991	validation_1-rmse:3768.09473
[4]	validation_0-rmse:3379.55786	validation_1-rmse:3411.32959
[5]	validation_0-rmse:3062.04761	validation_1-rmse:3091.94897
[6]	validation_0-rmse:2780.25049	validation_1-rmse:2808.51709
[7]	validation_0-rmse:2523.37280	validation_1-rmse:2548.63403
[8]	validation_0-rmse:2294.15625	validation_1-rmse:2318.70068
[9]	validation_0-rmse:2088.35547	validation_1-rmse:2111.89844
[10]	validation_0-rmse:1904.20691	validation_1-rmse:1927.49963
[11]	validation_0-rmse:1739.56604	validation_1-rmse:1762.07495
[12]	validation_0-rmse:1593.12622	validation_1-rmse:1615.31433
[13]	validation_0-rmse:1462.44299	validation_1-rmse:1484.15833
[14]	validation_0-rmse:1345.76209	validation_1-rmse:1367.04626
[15]	validation_0-rmse:1242.53711	validation_1-rmse:1263.49853
[1

[134]	validation_0-rmse:419.26090	validation_1-rmse:533.54822
[135]	validation_0-rmse:418.95090	validation_1-rmse:533.59351
[136]	validation_0-rmse:418.69794	validation_1-rmse:533.35706
[137]	validation_0-rmse:417.69144	validation_1-rmse:533.30212
[138]	validation_0-rmse:416.66708	validation_1-rmse:533.15698
[139]	validation_0-rmse:415.97989	validation_1-rmse:533.04138
[140]	validation_0-rmse:415.89926	validation_1-rmse:532.98627
[141]	validation_0-rmse:415.19949	validation_1-rmse:532.89990
[142]	validation_0-rmse:414.33606	validation_1-rmse:532.73041
[143]	validation_0-rmse:413.47339	validation_1-rmse:532.54199
[144]	validation_0-rmse:412.76370	validation_1-rmse:532.36707
[145]	validation_0-rmse:412.49332	validation_1-rmse:532.34259
[146]	validation_0-rmse:412.15976	validation_1-rmse:532.41858
[147]	validation_0-rmse:411.13626	validation_1-rmse:532.56348
[148]	validation_0-rmse:410.52698	validation_1-rmse:532.35809
[149]	validation_0-rmse:409.67520	validation_1-rmse:532.18915
[150]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
%%time
# Model predictions
y_pred_val = model.predict(X_val)

CPU times: user 78.2 ms, sys: 3.33 ms, total: 81.5 ms
Wall time: 15.3 ms


In [22]:
%%time
# Model predictions
y_pred_train = model.predict(X_train)

CPU times: user 225 ms, sys: 4.45 ms, total: 230 ms
Wall time: 38.3 ms


## Training set error

In [23]:
# Model predictions
rmse_train = mean_squared_error(y_train, y_pred_train)**0.5
rmse_train

389.37212207343407

In [24]:
mae_train=mean_absolute_error(y_train, y_pred_train)
mae_train

222.8582586039278

In [25]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9827284674918583

## Model validation

In [26]:
#432
rmse_val = mean_squared_error(y_val, y_pred_val)**0.5
rmse_val

530.3426993785816

In [27]:
mae_val=mean_absolute_error(y_val, y_pred_val)
mae_val

272.2251012771714

In [28]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9827284674918583

## Model training without validation

In [29]:
%%time
# Model training
model.fit(X, y,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:5054.22119	validation_1-rmse:5090.10205
[1]	validation_0-rmse:4565.72168	validation_1-rmse:4599.03418
[2]	validation_0-rmse:4127.29590	validation_1-rmse:4158.41797
[3]	validation_0-rmse:3733.60474	validation_1-rmse:3761.63818
[4]	validation_0-rmse:3377.64062	validation_1-rmse:3405.09814
[5]	validation_0-rmse:3059.72754	validation_1-rmse:3083.68677
[6]	validation_0-rmse:2777.68213	validation_1-rmse:2799.39990
[7]	validation_0-rmse:2521.07056	validation_1-rmse:2540.46973
[8]	validation_0-rmse:2290.52710	validation_1-rmse:2308.39111
[9]	validation_0-rmse:2084.54394	validation_1-rmse:2099.37012
[10]	validation_0-rmse:1900.79175	validation_1-rmse:1913.46375
[11]	validation_0-rmse:1736.11133	validation_1-rmse:1746.84106
[12]	validation_0-rmse:1589.89734	validation_1-rmse:1598.46631
[13]	validation_0-rmse:1459.55493	validation_1-rmse:1466.12158
[14]	validation_0-rmse:1344.21069	validation_1-rmse:1349.36377
[15]	validation_0-rmse:1242.22571	validation_1-rmse:1244.74548
[1

[134]	validation_0-rmse:430.64477	validation_1-rmse:426.88071
[135]	validation_0-rmse:430.04926	validation_1-rmse:426.53229
[136]	validation_0-rmse:429.52838	validation_1-rmse:425.68100
[137]	validation_0-rmse:429.20254	validation_1-rmse:425.42429
[138]	validation_0-rmse:428.41260	validation_1-rmse:424.33322
[139]	validation_0-rmse:427.82669	validation_1-rmse:423.87784
[140]	validation_0-rmse:427.41385	validation_1-rmse:423.30457
[141]	validation_0-rmse:426.94125	validation_1-rmse:422.97797
[142]	validation_0-rmse:426.49176	validation_1-rmse:422.44080
[143]	validation_0-rmse:426.01621	validation_1-rmse:422.15793
[144]	validation_0-rmse:424.61948	validation_1-rmse:420.50372
[145]	validation_0-rmse:424.28659	validation_1-rmse:420.30728
[146]	validation_0-rmse:423.28110	validation_1-rmse:419.37802
[147]	validation_0-rmse:422.36768	validation_1-rmse:418.96976
[148]	validation_0-rmse:421.75263	validation_1-rmse:418.21387
[149]	validation_0-rmse:421.44272	validation_1-rmse:417.90008
[150]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Test Preditions

In [30]:
predictions = model.predict(X_test)

In [31]:
predictions=pd.DataFrame(predictions)

In [32]:
predictions.reset_index(inplace=True)

In [33]:
predictions=predictions.rename({0: 'price','index': 'id'}, axis=1)

## Save Preditions

In [34]:
predictions.to_csv('../data/diamonds_predictions_catcodes_cross_target.csv',index=False)